## 평가 데이터셋

In [1]:
import os
import json
import time
from dotenv import load_dotenv
from tqdm import tqdm
from textwrap import dedent

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()

True

In [2]:
scenarios = [
    {
        "relationship_stage": "헤어진 지 한 달",
        "conflict_situation": "이별 후에도 전 연인의 SNS를 계속 확인하며 감정에 휘둘리는 상황",
        "question_tone": "무기력하고 체념 섞인 톤",
        "format_condition": "감정과 행동의 괴리를 반성적으로 묘사"
    },
    {
        "relationship_stage": "연애 중",
        "conflict_situation": "상대가 사랑한다면서도 늘 우선순위는 일이거나 친구인 상황",
        "question_tone": "조용히 서운한 톤",
        "format_condition": "일상 예시 중심의 서술 포함"
    },
    {
        "relationship_stage": "썸 타는 중",
        "conflict_situation": "상대가 매력적이지만, 나에게 진심인지 확신이 없는 상황",
        "question_tone": "분석적이고 거리두는 시선",
        "format_condition": "감정보다 관찰 중심의 전개"
    },
    {
        "relationship_stage": "이별 직전",
        "conflict_situation": "애정은 남았지만 서로 말이 통하지 않는 갈등 지속",
        "question_tone": "혼란과 체념이 섞인 톤",
        "format_condition": "대화 예시와 반복되는 패턴 서술 포함"
    },
    {
        "relationship_stage": "연애 중",
        "conflict_situation": "상대의 불안정한 감정 기복에 휘둘리는 상황",
        "question_tone": "관찰적이며 자기방어적인 톤",
        "format_condition": "상대와 자신의 반응 비교 서술"
    },
    {
        "relationship_stage": "짝사랑 중",
        "conflict_situation": "계속 마음이 커지는데 상대는 무심한 상황",
        "question_tone": "덤덤하고 자조적인 톤",
        "format_condition": "혼잣말처럼 전개되는 생각 흐름 포함"
    },
    {
        "relationship_stage": "헤어진 후 친구 유지 중",
        "conflict_situation": "전 연인이 새로운 연애를 시작했지만, 여전히 연락하는 상황",
        "question_tone": "복잡한 감정의 객관화 시도",
        "format_condition": "역할의 모호함에 대한 통찰 포함"
    },
    {
        "relationship_stage": "결혼 준비 중",
        "conflict_situation": "경제적 현실과 감정의 거리감이 커지는 상황",
        "question_tone": "냉정하면서도 무거운 톤",
        "format_condition": "가치관 충돌 서술 포함"
    },
    {
        "relationship_stage": "연애 중",
        "conflict_situation": "자꾸 비교당하는 기분이 드는 연애",
        "question_tone": "서늘한 의심과 자기 성찰이 섞인 톤",
        "format_condition": "은유적 표현 포함"
    },
    {
        "relationship_stage": "짝사랑 종료",
        "conflict_situation": "상대가 연애를 시작하며 물리적 거리도 멀어진 상황",
        "question_tone": "담담한 체념",
        "format_condition": "스스로를 돌아보는 결론 포함"
    },
    {
        "relationship_stage": "동거 중",
        "conflict_situation": "생활 습관 차이로 점점 지쳐가는 상황",
        "question_tone": "피곤함과 회의감이 공존",
        "format_condition": "관찰적 서술과 현실 분석 포함"
    },
    {
        "relationship_stage": "헤어진 지 오래된 상황",
        "conflict_situation": "문득 생각나는 그 사람에게 아직 감정이 남아 있는 듯한 순간",
        "question_tone": "노스탤지어적 감정과 이성의 혼합",
        "format_condition": "시간의 흐름과 감정의 변화 포함"
    },
    {
        "relationship_stage": "첫 연애",
        "conflict_situation": "모든 게 낯설고 서툴러 관계가 불안정한 상황",
        "question_tone": "어색하고 불안한 감정",
        "format_condition": "스스로에 대한 질문 포함"
    },
    {
        "relationship_stage": "이별 후",
        "conflict_situation": "나를 쉽게 놓고 간 그 사람에게서 여전히 벗어나지 못하는 상황",
        "question_tone": "혼란보다 허탈에 가까운 톤",
        "format_condition": "자기기만에 대한 인식 포함"
    },
    {
        "relationship_stage": "썸 타는 중",
        "conflict_situation": "상대의 관심이 나에게서 다른 곳으로 옮겨가는 느낌",
        "question_tone": "관찰적이고 차가운 분석",
        "format_condition": "메시지 흐름 분석 포함"
    },
    {
        "relationship_stage": "연애 중",
        "conflict_situation": "갈등이 생길 때마다 감정을 피하려는 상대의 태도",
        "question_tone": "답답함보다 체념적 인내",
        "format_condition": "대화 흐름 묘사 포함"
    },
    {
        "relationship_stage": "헤어진 후 몇 달 경과",
        "conflict_situation": "이젠 나아졌다고 생각했는데, 특정 상황에서 무너지는 자신",
        "question_tone": "불쑥 올라온 감정에 당황한 톤",
        "format_condition": "예기치 않은 자극과 반응 포함"
    },
    {
        "relationship_stage": "재회 시도 중",
        "conflict_situation": "예전과 똑같은 문제로 다시 다투는 상황",
        "question_tone": "지침과 익숙함이 공존",
        "format_condition": "패턴 분석 포함"
    },
    {
        "relationship_stage": "장거리 연애",
        "conflict_situation": "연락은 되지만 감정 교류가 점점 희미해지는 상황",
        "question_tone": "이해하려 애쓰는 시선",
        "format_condition": "상호 노력의 균형 분석 포함"
    },
    {
        "relationship_stage": "소개팅 후 연락 중",
        "conflict_situation": "서로 호감은 있는데 계속 타이밍이 엇갈리는 상황",
        "question_tone": "체념 섞인 유머",
        "format_condition": "‘이게 타이밍 문제일까 사람 문제일까’ 시선 포함"
    },
    {
        "relationship_stage": "연애 중",
        "conflict_situation": "상대가 내 감정을 잘 모른다고 느끼는 상황",
        "question_tone": "소외감과 객관화 시도",
        "format_condition": "자신의 표현 방식에 대한 분석 포함"
    },
    {
        "relationship_stage": "짝사랑 중",
        "conflict_situation": "상대에게는 그냥 친한 친구일 뿐인 자신",
        "question_tone": "담담한 서글픔",
        "format_condition": "관계의 위치 분석 포함"
    },
    {
        "relationship_stage": "연애 중",
        "conflict_situation": "상대가 지나치게 감정적으로 대처하는 문제",
        "question_tone": "이성적인 거리를 유지하려는 태도",
        "format_condition": "서로 감정의 온도차 비교 포함"
    },
    {
        "relationship_stage": "결혼 이후",
        "conflict_situation": "연애 때와 달라진 태도에 실망한 상황",
        "question_tone": "냉정하고 체념한 시선",
        "format_condition": "기대 vs 현실 비교 포함"
    },
    {
        "relationship_stage": "썸 타는 중",
        "conflict_situation": "상대가 계속해서 확신을 주지 않는 상황",
        "question_tone": "덧없음에 가까운 무덤덤한 감정",
        "format_condition": "기다림의 무게 표현 포함"
    },
    {
        "relationship_stage": "연애 중",
        "conflict_situation": "나에겐 중요한 기념일인데 상대는 잊고 지나친 상황",
        "question_tone": "담담한 실망",
        "format_condition": "기억의 불균형에 대한 관찰 포함"
    },
    {
        "relationship_stage": "헤어진 후",
        "conflict_situation": "상대는 금방 연애를 시작했는데 나는 여전히 정체된 상황",
        "question_tone": "비교로 인한 자기 혐오 감정",
        "format_condition": "현실과 감정 사이 간극 묘사 포함"
    },
    {
        "relationship_stage": "연애 중",
        "conflict_situation": "상대가 노력하지 않아도 계속 관계가 유지된다고 생각하는 듯한 태도",
        "question_tone": "피로감과 성찰",
        "format_condition": "기울어진 노력의 비유 포함"
    },
    {
        "relationship_stage": "헤어진 직후",
        "conflict_situation": "서로 헤어지자고 해놓고 연락을 이어가는 상황",
        "question_tone": "모순을 인식하면서도 끊지 못하는 톤",
        "format_condition": "이중 감정 서술 포함"
    },
    {
        "relationship_stage": "첫 연애",
        "conflict_situation": "사랑을 잘 못 믿게 되는 자신을 발견한 상황",
        "question_tone": "조심스러운 자기 분석",
        "format_condition": "감정의 뿌리 탐색 포함"
    }
]


In [3]:
prompt_template = dedent("""
# ROLE
당신은 AI 연애 상담 모델의 성능을 평가하기 위한 고품질 테스트 QA 데이터셋을 생성하는 AI 전문가입니다.

# GOAL
주어진 시나리오와 페르소나를 바탕으로, 현실적이고 다채로운 질문(instruction)과 그에 대한 이상적인 답변(output) 쌍을 1개 생성합니다. 출력은 반드시 JSON 배열 형식이어야 합니다.

# PERSONA FOR THE AI STREAMER(for OUTPUT)
- 말투: 담담하지만 무심하지 않음. 지나치게 감정적이지 않으면서도 핵심을 찌르는 톤, 때때로 자조 섞인 유머를 사용하며 상대방을 비웃기보다 같이 허탈해함, 구어체지만 문장은 정돈되어 있음. (예: “그건 슬프죠. 하지만 그게 현실이에요.”)
- 화법: 논리 + 관찰 + 경험이 결합된 분석형 화법, "내가 너보다 낫다"는 위계 없음. 오히려 “나도 너랑 다르지 않다”는 포지셔닝, 감정 대신 맥락을 짚음. “왜 그랬는지 이해는 된다. 그런데 말이야…”식 접근, 말하는 도중 한숨 혹은 체념적 정리를 넣어 인간적인 매듭을 줌.
- 스타일: 연애를 소재로 현실 사회관찰로까지 넓힘 (외모지상주의, 책임, 희생 등). 스토리텔링이 아니라 “현상 분석+개인 통찰”로 결론을 이끎, 실명/정확한 정보는 피하고 사례-상징-은유 중심
- 핵심 철학: “사랑은 비합리적이다.”, “우리는 늘 계산하며 사랑하지만, 진짜 사랑은 계산이 아니다.”, “내가 이득을 얻는 연애가 아니라, 손해를 감수하고도 하고 싶은 연애가 진짜다.”, “질투도, 욕망도, 모순도 인간이다. 그걸 받아들이는 게 어른이다.”
                  
# SCENARIO & CONDITIONS
- 관계 단계: {relationship_stage}
- 갈등 상황: {conflict_situation}
- 질문 톤: {question_tone}
- 형식: {format_condition}

# OUTPUT FORMAT
- 반드시 아래와 같은 JSON 배열 형식으로만 응답해야 합니다.
- 'instruction'은 시청자의 입장에서 작성된 구체적인 질문이어야 합니다.
- 'output'은 주어진 페르소나의 원칙과 말투를 완벽하게 재현한 답변이어야 합니다.
[
  {{
    "instruction": "<위 시나리오에 맞는 사용자의 질문>",
    "input": "",
    "output": "<위 페르소나 원칙에 맞는 이상적인 답변>"
  }}
]
                         
---

# DEMONSTRATIONS (Few-shot Examples):

### 예시 1
"instruction": "남자친구의 여자 후배가 신경 쓰이는데 제가 예민한 걸까요?",
"input": ""
"output": "예민한 거 맞아요. 근데 그게 나쁘다는 건 아닙니다. 질투는 비이성적이죠. 사랑도 그래요. 다만, 상대는 그걸 해명할 수 없고, 당신은 근거를 제시할 수 없기 때문에 다툼은 소모적이 됩니다. ‘불편하다’는 감정을 탓하지 말고, ‘해결할 수 없는 감정’임을 인정해 주세요."

### 예시 2
"instruction": "예쁜 여자만 대접받는 세상이 너무 억울하게 느껴져요.",
"input": "",
"output": "네. 슬픈 이야기죠. 근데 이건 개인의 문제가 아니라 사회의 구조 문제입니다. 예쁨이 권력이 된 사회에서, 외모는 배려를 결정짓는 요소가 되니까요. 당신이 느끼는 모멸감은 유효해요. 다만, 그 구조를 바꿀 순 없어도, 거기에 자존감을 맡길 필요는 없습니다."    
                           
### 예시 3
"instruction": "썸만 탔던 사람이 결혼했는데 허무한 기분이 들어요. 왜 그럴까요?",
"input": "4년 전에 썸만 타고 끝난 사람의 결혼 소식을 봤어요. 왜 이렇게 허무할까요?",
"output": "허무하죠. 사귄 것도 아니었는데, 왜 허무할까요? 그 사람은 내 인생에 아무 일도 안 했고, 나는 그 사람 인생에 아무것도 아니었는데. 그냥 ‘예뻤던 가능성’이 사라졌기 때문이에요. 우리는 때로 실재보다 가능성에 더 감정소모를 하니까요."
""")

In [4]:
def generate_test_data_langchain(scenario):
    """하나의 시나리오를 받아 QA-Pair를 생성합니다."""
    try:
        prompt = ChatPromptTemplate.from_template(prompt_template)
        model = ChatOpenAI(model="gpt-4.1", temperature=0.8)
        parser = JsonOutputParser()
        chain = prompt | model | parser
        result = chain.invoke({
            "relationship_stage": scenario["relationship_stage"],
            "conflict_situation": scenario["conflict_situation"],
            "question_tone": scenario["question_tone"],
            "format_condition": scenario["format_condition"]
        })
        
        # 파서가 반환한 딕셔너리에서 'qa_pairs' 리스트를 추출
        return result

    except Exception as e:
        print(f"'{scenario['conflict_situation']}' 시나리오 처리 중 LangChain 오류 발생: {e}")
        return None

# --- 메인 실행 로직 ---
all_test_data = []
output_filename = "omar_test_dataset.json"

print("테스트 데이터셋 생성을 시작합니다...")
for scenario in tqdm(scenarios, desc="시나리오 처리 중"):
    print(f"\n'{scenario['relationship_stage']}' 시나리오 생성 중...")
    generated_pairs = generate_test_data_langchain(scenario)
    
    if generated_pairs and isinstance(generated_pairs, list):
        all_test_data.extend(generated_pairs)
        print(f"✅ {len(generated_pairs)}개의 QA 쌍 생성 완료. (총 {len(all_test_data)}개)")
    else:
        print("❌ 데이터 생성에 실패했습니다.")
    
    time.sleep(5)

# --- 최종 파일 저장 ---
try:
    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(all_test_data, f, ensure_ascii=False, indent=2)
    print(f"\n🎉 성공! 총 {len(all_test_data)}개의 테스트 데이터가 '{output_filename}' 파일에 저장되었습니다.")
except Exception as e:
    print(f"🚨 최종 파일 저장 중 오류 발생: {e}")

테스트 데이터셋 생성을 시작합니다...


시나리오 처리 중:   0%|          | 0/30 [00:00<?, ?it/s]


'헤어진 지 한 달' 시나리오 생성 중...
✅ 1개의 QA 쌍 생성 완료. (총 1개)


시나리오 처리 중:   3%|▎         | 1/30 [00:11<05:42, 11.80s/it]


'연애 중' 시나리오 생성 중...
✅ 1개의 QA 쌍 생성 완료. (총 2개)


...


'헤어진 직후' 시나리오 생성 중...
✅ 1개의 QA 쌍 생성 완료. (총 29개)


시나리오 처리 중:  97%|█████████▋| 29/30 [05:15<00:10, 10.03s/it]


'첫 연애' 시나리오 생성 중...
✅ 1개의 QA 쌍 생성 완료. (총 30개)


시나리오 처리 중: 100%|██████████| 30/30 [05:25<00:00, 10.84s/it]


🎉 성공! 총 30개의 테스트 데이터가 'omar_test_dataset.json' 파일에 저장되었습니다.
